<a href="https://colab.research.google.com/github/koksal100/Deep-Learning/blob/main/AnadoluHayat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sigorta/train.csv")

<ipython-input-2-9e78fc376c58>:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sigorta/train.csv")


In [ ]:
df.columns

Index(['MUSTERI_ID', 'LABEL', 'FLAG', 'PP_CINSIYET', 'PP_YAS', 'PP_MESLEK',
       'PP_MUSTERI_SEGMENTI', 'PP_UYRUK', 'IL', 'SORU_YATIRIM_KARAKTERI_CVP',
       'SORU_YATIRIM_KARAKTERI_RG', 'SORU_MEDENI_HAL_CVP',
       'SORU_MEDENI_HAL_RG', 'SORU_EGITIM_CVP', 'SORU_EGITIM_RG',
       'SORU_GELIR_CVP', 'SORU_GELIR_RG', 'SORU_COCUK_SAYISI_CVP',
       'SORU_COCUK_SAYISI_RG', 'BES_AYRILMA_TALEP_ADET', 'ODEMEME_TALEP_ADET',
       'HAYAT_AYRILMA_TALEP_ADET', 'BILGI_TALEP_ADET', 'VADE_TUTAR_0',
       'ODEME_TUTAR_0', 'VADE_TUTAR_1', 'ODEME_TUTAR_1', 'VADE_TUTAR_2',
       'ODEME_TUTAR_2', 'VADE_TUTAR_3', 'ODEME_TUTAR_3', 'VADE_TUTAR_4',
       'ODEME_TUTAR_4', 'VADE_TUTAR_5', 'ODEME_TUTAR_5', 'VADE_TUTAR_6',
       'ODEME_TUTAR_6', 'VADE_TUTAR_7', 'ODEME_TUTAR_7', 'VADE_TUTAR_8',
       'ODEME_TUTAR_8', 'VADE_TUTAR_9', 'ODEME_TUTAR_9', 'VADE_TUTAR_10',
       'ODEME_TUTAR_10', 'VADE_TUTAR_11', 'ODEME_TUTAR_11',
       'SON_AY_KATKI_MIKTARI', 'SON_AY_KATKI_ADET', 'SON_CEYREK_KATKI_MIKTARI'

In [ ]:
df.drop("MUSTERI_ID",axis=1,inplace=True)

In [ ]:
df["LABEL"]

0           UA
1           UA
2         HU19
3           UA
4           UA
          ... 
852714      UA
852715      UA
852716      UA
852717      UA
852718      UA
Name: LABEL, Length: 852719, dtype: object

In [ ]:
df=pd.concat([pd.get_dummies(df["LABEL"],prefix="label"),df],axis=1)

df.drop("LABEL",axis=1,inplace=True)

df["SON_SENE_KATKI_MIKTARI"].describe()

corelations=df.corr(numeric_only=True)[["label_HU06","label_HU07","label_HU11","label_HU12","label_HU14","label_HU15","label_HU19","label_UA"]]

df.select_dtypes(include="object")

corelations.loc["SON_SENE_KATKI_MIKTARI"]

meslekGroupby=df.groupby("PP_MESLEK").mean(numeric_only=True)[["label_HU06","label_HU07","label_HU11","label_HU12","label_HU14","label_HU15","label_HU19"]]
meslekGroupby.loc["Akademisyen"].argmax()

karaktergroupby=df.groupby("SORU_YATIRIM_KARAKTERI_CVP").mean(numeric_only=True)[["label_HU06","label_HU07","label_HU11","label_HU12","label_HU14","label_HU15","label_HU19"]]
karaktergroupby

medeniGroupby=df.groupby("SORU_MEDENI_HAL_CVP").mean(numeric_only=True)[["label_HU06","label_HU07","label_HU11","label_HU12","label_HU14","label_HU15","label_HU19"]]
medeniGroupby

eğitimGroupby=df.groupby("SORU_EGITIM_CVP").mean(numeric_only=True)[["label_HU06","label_HU07","label_HU11","label_HU12","label_HU14","label_HU15","label_HU19"]]
eğitimGroupby

df["MeslekTercih"]=df["PP_MESLEK"].apply(lambda x:meslekGroupby.columns[meslekGroupby.loc[x].argmax()] if x in meslekGroupby.index else "label_UA")

df["KarakterTercih"]=df["SORU_YATIRIM_KARAKTERI_CVP"].apply(lambda x:karaktergroupby.columns[karaktergroupby.loc[x].argmax()] if x in karaktergroupby.index else "label_UA")

df["MedeniTercih"]=df["SORU_MEDENI_HAL_CVP"].apply(lambda x:medeniGroupby.columns[medeniGroupby.loc[x].argmax()] if x in medeniGroupby.index else "label_UA")

df["EğitimTercih"]=df["SORU_EGITIM_CVP"].apply(lambda x:eğitimGroupby.columns[eğitimGroupby.loc[x].argmax()] if x in eğitimGroupby.index else "label_UA")


df.drop("PP_MESLEK",axis=1,inplace=True)

df=pd.concat([df,pd.get_dummies(df["FLAG"])],axis=1)

df.drop("FLAG",axis=1,inplace=True)


df.isnull().sum().sort_values(ascending=False).head(30)

df.select_dtypes(include="object").columns

df.drop("SORU_YATIRIM_KARAKTERI_CVP",axis=1,inplace=True)

df.drop("SORU_MEDENI_HAL_CVP",axis=1,inplace=True)

df.drop("SORU_EGITIM_CVP",axis=1,inplace=True)

def convert(x):
    if isinstance(x,float):
        return x
    if "," in x:
        x=x.replace(",",".")
    if isinstance(x,float):
        return x
    else:
        return float(x)

df["SORU_GELIR_CVP"]=df["SORU_GELIR_CVP"].apply(convert)

df["SORU_GELIR_CVP"].fillna(df["SORU_GELIR_CVP"].mean(),inplace=True)

df["SORU_GELIR_CVP"]

df.select_dtypes(include="object").columns

df.isnull().sum().sort_values(ascending=False).head(20)

df.drop("SORU_MEDENI_HAL_RG",axis=1,inplace=True)

df.drop("SORU_EGITIM_RG",axis=1,inplace=True)

df.drop("SORU_GELIR_RG",axis=1,inplace=True)

df["SORU_COCUK_SAYISI_CVP"].value_counts()

0.0       411051
2.0        10765
1.0        10240
3.0         2925
4.0          757
5.0          259
6.0           90
7.0           36
8.0           19
10.0           8
9.0            3
11.0           2
3000.0         1
14.0           1
20.0           1
21.0           1
Name: SORU_COCUK_SAYISI_CVP, dtype: int64

In [ ]:
df["SORU_COCUK_SAYISI_CVP"].fillna(df["SORU_COCUK_SAYISI_CVP"].mean(),inplace=True)

df.drop("SORU_COCUK_SAYISI_RG",axis=1,inplace=True)

df.drop("SORU_YATIRIM_KARAKTERI_RG",axis=1,inplace=True)

In [ ]:
df["SON_AY_KATKI_ADET"].fillna(df["SON_AY_KATKI_ADET"].mean(),inplace=True)

df["SON_AY_KATKI_ADET"].fillna(df["SON_AY_KATKI_ADET"].mean(),inplace=True)

df["SON_AY_KATKI_MIKTARI"].fillna(df["SON_AY_KATKI_MIKTARI"].mean(),inplace=True)

df["SON_CEYREK_KATKI_ADET"].fillna(df["SON_CEYREK_KATKI_ADET"].mean(),inplace=True)

df["SON_CEYREK_KATKI_MIKTARI"].fillna(df["SON_CEYREK_KATKI_MIKTARI"].mean(),inplace=True)

df["SON_SENE_KATKI_ADET"].fillna(df["SON_SENE_KATKI_ADET"].mean(),inplace=True)

df["SON_SENE_KATKI_MIKTARI"].fillna(df["SON_SENE_KATKI_MIKTARI"].mean(),inplace=True)

from sklearn.linear_model import LogisticRegression

In [ ]:
for column in df.select_dtypes(include="object").columns:
    df=pd.concat([pd.get_dummies(df[column]),df],axis=1)
    df.drop(column,axis=1,inplace=True)




In [ ]:
classWeights={
   "HU06" : 0.0385,
"HU07" : 0.0328,
"HU11" : 0.2791,
"HU12" : 0.1812,
"HU14" : 0.0113,
"HU15" : 0.2952,
"HU19" : 0.1614,
"UA" : 0.0001 ,
}
x=df.drop(["label_HU06","label_HU07","label_HU11","label_HU12","label_HU14","label_HU15","label_HU19","label_UA"],axis=1)

y=df[["label_HU06","label_HU07","label_HU11","label_HU12","label_HU14","label_HU15","label_HU19","label_UA"]]

df1=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sigorta/train.csv")
y=df1["LABEL"]



In [ ]:
model = LogisticRegression(multi_class='ovr',class_weight=classWeights,max_iter=500000)
model.fit(x, y)



from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

from sklearn.metrics import f1_score

f1_score(y_test,model.predict(X_test),average="macro")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

from sklearn.metrics import f1_score

st=StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
X_train=st.fit_transform(X_train)
X_test=st.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier(n_neighbors=8)


knn.fit(X_train, y_train)

# Test kümesi üzerinde tahmin yap
y_pred = knn.predict(X_test)

f1_score(y_test,y_pred,average="macro")

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Modeli oluşturma
model = DecisionTreeClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)

f1_score(y_test,y_pred,average="macro")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Modeli oluşturma
model = RandomForestClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")

In [ ]:
from sklearn.svm import SVC

# Modeli oluşturma
model = SVC(decision_function_shape='ovo')

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")

In [ ]:
from sklearn.neural_network import MLPClassifier

# Modeli oluşturma
model = MLPClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Modeli oluşturma
model = GradientBoostingClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)

f1_score(y_test,y_pred,average="macro")


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# Modeli oluşturma
model = AdaBoostClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")

In [ ]:
from xgboost import XGBClassifier

# Modeli oluşturma
model = XGBClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")

In [ ]:

from lightgbm import LGBMClassifier

# Modeli oluşturma
model = LGBMClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")

In [ ]:

from catboost import CatBoostClassifier

# Modeli oluşturma
model = CatBoostClassifier()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")


In [ ]:
from sklearn.naive_bayes import GaussianNB

# Modeli oluşturma
model = GaussianNB()

# Veriyi modele uydurma
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)


f1_score(y_test,y_pred,average="macro")








